In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

import transformers as ppb # pytorch-pretrained-bert
import torch

import pipeline as pp
import models as ml

import importlib
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\madsv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
importlib.reload(pp)
from_file = "../datasets/big/dataset_bin.csv"
BATCH_SIZE = 5000

pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [False, False, False], 
                                   out_file="../datasets/sample/dataset_unbalanced.csv", get_frame=False)

100%|██████████| 200000/200000 [00:00<00:00, 805975.39it/s]


entries read: 200000


In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

# Create a corpus of text data
corpus = ["I like apples", "I love oranges", "She hates pears", "He dislikes bananas"]

# Create a dictionary of words and their indices
word_dict = {}
for sentence in corpus:
    for word in sentence.split():
        if word not in word_dict:
            word_dict[word] = len(word_dict)

print(word_dict)

# Convert the corpus into a matrix of word indices
corpus_matrix = np.zeros((len(corpus), len(word_dict)))
for i, sentence in enumerate(corpus):
    for word in sentence.split():
        corpus_matrix[i, word_dict[word]] = 1

print(corpus_matrix)

# Create a neural network model with an embedding layer
model = Sequential()
model.add(Embedding(len(word_dict), 10, input_length=len(word_dict)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(corpus_matrix, np.array([1, 1, 0, 0]), epochs=100, verbose=0)

# Get the learned word embeddings
embeddings = model.get_weights()[0]

# Print the learned embeddings for each word
for word, index in word_dict.items():
    print(word, embeddings[index])


{'I': 0, 'like': 1, 'apples': 2, 'love': 3, 'oranges': 4, 'She': 5, 'hates': 6, 'pears': 7, 'He': 8, 'dislikes': 9, 'bananas': 10}
[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
I [ 0.06881227 -0.13503245  0.13067168  0.16277544  0.1175928   0.14362365
 -0.05564805  0.05248439  0.08392346 -0.15577602]
like [-0.14159463  0.15149534 -0.14801809 -0.1548039  -0.07514571 -0.1058769
  0.05275943 -0.07659105 -0.15096149  0.1543007 ]
apples [-0.0305975  -0.04509356  0.03028085  0.01999826 -0.00269739 -0.00310541
  0.02410186 -0.04725834  0.02400244  0.04329363]
love [-0.04896925 -0.00796487 -0.04935968 -0.01651973 -0.0330081  -0.00728096
  0.00932461  0.00094824  0.04020016  0.00554812]
oranges [-0.01422012 -0.01638101 -0.03042551  0.00591964 -0.01169596  0.04766793
 -0.02327204 -0.00236449  0.01989326  0.02445111]
She [-0.00154535 -0.03032771  0.00611943 -0.0191129  -0.04995989 -0.01201131
 -0.03

In [ ]:
# Load the dataset
df = pd.read_csv('fake_news_dataset.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Convert the text data to sequences of word indices
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a fixed length
max_len = 1000
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Create the word embedding matrix
word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Create the neural network model
model = Sequential()
model.add(Embedding(len(word_index)+1, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
score = model.evaluate(X_test_pad, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
